In [1]:
import pyrosetta 
import rosetta 
from sys import argv
import os 

In [2]:
with open( '../../mutant_list.txt' ) as fn:
    mutant_list = fn.read().split()

os.environ[ 'SLURM_ARRAY_TASK_ID' ] = '39' # this is for development only 
mutant_name = mutant_list[ int( os.environ[ 'SLURM_ARRAY_TASK_ID' ] ) ] 

print( mutant_name ) 

fmt = dict( zip( 'ANDRCQEGHILKMPFSTWYV', [
    'ALA','ASN','ASP','ARG','CYS','GLN','GLU',
    'GLY','HIS','ILE','LEU','LYS','MET','PRO','PHE','SER',
    'THR','TRP','TYR','VAL' ] ) ) 

with open( 'input_files/flags' ) as fn:
    flags = fn.read().replace( '\n', ' ' )

# init PyRosetta 
pyrosetta.init( ''.join( flags ) ) 

ligand_params = pyrosetta.Vector1( [ 'input_files/pNPG.params' ] )
new_res_set = pyrosetta.generate_nonstandard_residue_set( ligand_params )

p = pyrosetta.Pose()
pyrosetta.pose_from_file( p, new_res_set, 'input_files/bglb.pdb' ) 
scorefxn = pyrosetta.create_score_function( 'beta_cst' ) 

add_cst = rosetta.protocols.enzdes.AddOrRemoveMatchCsts()
add_cst.cstfile( 'input_files/pNPG.enzdes.cst' ) 
add_cst.set_cst_action( rosetta.protocols.enzdes.CstAction.ADD_NEW )
add_cst.apply( p ) 

target = int( mutant_name[ 1:-1 ] )
new_res = fmt[ mutant_name[ -1 ] ] 
mut = rosetta.protocols.simple_moves.MutateResidue( target, new_res )
mut.apply( p ) 

E406V
Found rosetta database at: /usr/local/bin/anaconda3/lib/python3.5/site-packages/pyrosetta-4.0-py3.5.egg/database; using it....
PyRosetta-4 2016 [Rosetta 2016 unknown:7862c3a14bb5e8d6b2a4c3e0a497795bc06e9e4b 2016-12-13 14:39:52 -0500] retrieved from: git@github.com:RosettaCommons/main.git
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.



In [3]:
# set up packing task 
tf = rosetta.core.pack.task.TaskFactory()
around = rosetta.protocols.toolbox.task_operations.DesignAroundOperation()
around.include_residue( 446 ) #ligand 
around.include_residue( target ) 
around.repack_shell( 24 ) # let's grid search this  
around.resnums_allow_design( False )
tf.push_back( around ) 
pt = tf.create_task_and_apply_taskoperations(p)

# repack and minimize 
repack = rosetta.protocols.enzdes.EnzRepackMinimize()
repack.set_scorefxn_repack( scorefxn )
repack.set_scorefxn_minimize( scorefxn )
repack.set_min_bb( True )
repack.set_min_lig( True )  
repack.set_min_rb( True ) 
repack.set_min_sc( True )
repack.task_factory( tf ) # adds packer task 

# monte carlo 
parsed = rosetta.protocols.simple_moves.GenericMonteCarloMover()
parsed.set_mover( repack )
parsed.set_maxtrials( 10 )
parsed.set_scorefxn( scorefxn )
parsed.apply( p ) 

False

In [6]:
p.dump_pdb( 'output_files/{}.pdb'.format( mutant_name) ) 

True